# Process gridded observation data into timeseries

## 2024/08/20

Just do 5x5 data.

__1. Process the gridded temperature data into timeseries for each observational product.__

Output is a dataArray for each model with dimensions of time and IPCC region containing a time series of the TAS variable.


Use this tool:  

https://github.com/IPCC-WG1/Atlas/blob/main/notebooks/reference-regions_Python.ipynb

For now, I will create my code for the CESM1 and MPI models so that it can be generalized easily. I can pull some code from my climatetrend_uncertainty repository (climatetrend_uncertainty/initial_code/PIC_timeseries_preproc.ipynb).

## Code!

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob

import xagg as xa
import geopandas as gpd
import regionmask

regionmask.__version__

%matplotlib inline

__Observational Large Ensembles.__

General directory in Nathan's scratch: /glade/scratch/lenssen/data_for_jonah/

NASA GISTEMP: GISTEMP_2x2  GISTEMP_5x5

HadCRUT5: HadCRUT5

In [2]:
gistemp_2x2_dir = '/glade/derecho/scratch/lenssen/data4jonah/GISTEMP_Ensemble_Aug/' # I don't want to use this
gistemp_dir = "/glade/derecho/scratch/lenssen/data4jonah/GISTEMP_Ensemble_Aug_5x5/"
hadcrut5_dir = '/glade/derecho/scratch/lenssen/data4jonah/HadCRUT5_Ensemble_Aug/'
dcent_unfilled_dir = "/glade/u/home/jonahshaw/w/trend_uncertainty/nathan/OBS_LENS/DCENT/rawdata/DCENT_ensemble/"

### Collect file paths.

#### Collect GISTEMP file paths.

In [3]:
gistemp_files = glob.glob('%s/ensembleChunk_5x5_????.nc' % gistemp_dir)
gistemp_files.sort()

#### Collect HadCRUT5 file paths.

In [4]:
hadcrut5_files = glob.glob('%s/HadCRUT.5.0.2.0.analysis.anomalies.*.nc' % hadcrut5_dir)
hadcrut5_files.sort()

#### Collect DCENT (unfilled) file paths.

In [5]:
dcent_unfilled_files = glob.glob('%s/DCENT_ensemble_1850_2023_member_???.nc' % dcent_unfilled_dir)
dcent_unfilled_files.sort()

### Load and process timeseries according to IPCC Region designations.

Mask data based on availability.

### 2. Do masking for each dataset

Variable is "tempAnom". "record" coordinate will allow for easier concatenation.

In [6]:
gistemp_tas_var = 'tas'
hadcrut5_tas_var = 'tas'
dcent_unfiled_tas_var = "temperature"

### Loop over observation files and compute the regional means.

#### GISTEMP

In [7]:
def create_ipccregion_timeseries_xagg(
    ds_filepath:str,
    ds_var:str,
    model_str:str,
    cesm=False,
    read_wm=True,
    write_wm=True,
    new_times=None,
):
    
    '''
    Compute timeseries for all IPCC AR6 regions when given a simple model output file.
    Now using xagg to appropriately weight gridcells that fall partly within a region!
    '''
    # Load data
    ds = xr.open_dataset(ds_filepath)
    
    try:
        ds = ds.rename({"latitude":"lat", "longitude":"lon"})
    except:
        pass
    
    # Correct time if CESM
    if cesm:
        ds  = fix_cesm_time(ds)
    
    if new_times is not None:
        ds["time"] = new_times

    da = ds[ds_var]

    xagg_dir = "/glade/u/home/jonahshaw/w/trend_uncertainty/nathan/xagg_resources"
    xa.set_options(rgrd_alg='bilinear',nan_to_zero_regridding=False)

    if (read_wm and os.path.exists(os.path.join(xagg_dir, f'wm_{model_str}'))):
        # Load weightmap
        weightmap = xa.read_wm(os.path.join(xagg_dir, f'wm_{model_str}'))
    else:
        # Load IPCC region shp file:
        ipcc_wgi_regions_shp = "IPCC-WGI-reference-regions-v4.shp"
        gdf = gpd.read_file(os.path.join(xagg_dir, ipcc_wgi_regions_shp))
                
        # Compute weights for entire grid. Assuming lat, lon, time dimension on input
        area_weights = np.cos(np.deg2rad(da.lat)).broadcast_like(da.isel(time=0).squeeze())
        
        weightmap = xa.pixel_overlaps(da, gdf, weights=area_weights)
        # Save the weightmap for later:
        if write_wm:
            weightmap.to_file(os.path.join(xagg_dir, f'wm_{model_str}'))

    # Aggregate
    with xa.set_options(silent=True):
        aggregated = xa.aggregate(da, weightmap)
    # aggregated = xa.aggregate(da, weightmap)
    
    # Convert to an xarray dataset
    aggregated_ds = aggregated.to_dataset()
    # Change xarray formatting to match previous file organization.
    aggregated_ds = aggregated_ds.set_coords(("Continent", "Type", "Name", "Acronym")).rename({"poly_idx": "RegionIndex", "Name": "RegionName", "Acronym": "RegionAbbrev"})
        
    return aggregated_ds

In [8]:
def mask_IPCC_byavailability(
    filepath: str,
    var: str,
    regions: regionmask.Regions,
    masking_threshold: float=0.9,
    ufunc=None,
    new_times=None,
):
    """
    Function for masking observational record according to data availability.
    True on the output means that region exceeds the threshold.

    Args:
        filepath (str): Path to file to load
        var (str): String identifier of variable to mask by availabiliy
        masking_threshold (float, optional): Fraction of region needed to avoid masking. Defaults to 0.9.
        ufunc (function, optional): Function to apply to the loaded DataArray. Defaults to None.

    Returns:
        region_masks (xr.DataArray): Boolean mask for each IPCC region
    """

    ds = xr.open_dataset(filepath)

    da = ds[var]
    if ufunc is not None:
        da = ufunc(da)

    if new_times is not None:
        da["time"] = new_times

    mask    = regions.mask(da.isel(time=0))

    # Get unique region indices
    reg     = np.unique(mask.values)
    reg     = reg[~np.isnan(reg)]

    # for metadata: find abbreviations of all regions that were selected
    abbrevs = regions[reg].abbrevs
    names   = regions[reg].names

    # Compute weights for entire grid
    weights_spatial = np.cos(np.deg2rad(da.lat)).broadcast_like(da.isel(time=0)) # assuming 'lat' used consistently
    weights_spatiotemporal = np.cos(np.deg2rad(da.lat)).broadcast_like(da) # assuming 'lat' used consistently

    # Iterate over regions and compute a weighted time series
    region_masks_list = []
    for i,_abbrev,_name in zip(reg, abbrevs, names):
        
        _region_weight = weights_spatial.where(mask == i).sum(dim=["lat", "lon"]) # Total weight of the region
        _available_region_weights = weights_spatiotemporal.where((mask == i) & (~np.isnan(da))).sum(dim=["lat", "lon"]) # Weight of the unmasked portion of the region
        
        _region_mask = (_available_region_weights / _region_weight) > masking_threshold
        _region_mask = _region_mask.assign_coords(RegionIndex=i).expand_dims("RegionIndex")
        _region_mask = _region_mask.assign_coords(RegionName=_name)
        _region_mask = _region_mask.assign_coords(RegionAbbrev=_abbrev)
        
        region_masks_list.append(_region_mask)
    region_masks = xr.concat(region_masks_list, dim="RegionIndex")
    region_masks.name = "mask"
    
    return region_masks


def gistemp_5x5_preproc(ds):
    
    return ds.rename({"latitude":"lat", "longitude":"lon"})

In [9]:
save_dir = '/glade/u/home/jonahshaw/w/trend_uncertainty/nathan/OBS_LENS/'

#### GISTEMP

In [16]:
%%time

# gistemp_new_times = (pd.date_range("1900-01-01", freq="1M", periods=12*61)-pd.offsets.MonthBegin(1)).shift(periods=14,freq='D')
model_subdir = 'GISTEMP_5x5/20240820/xagg_correctedtime/'

if not os.path.exists(os.path.join(save_dir,model_subdir)):
    os.makedirs(os.path.join(save_dir,model_subdir))

# New time dimension to apply to correct drift from skipping leap years. JKS test.
new_times = pd.date_range('1880-01-16', '2020-12-31', freq='1ME') + pd.tseries.offsets.Day(-15)

# Variable to select and operate over.
_ds_var = gistemp_tas_var

for i,_ds_filepath in enumerate(gistemp_files):

    xagg_region_means = create_ipccregion_timeseries_xagg(
        ds_filepath=_ds_filepath,
        ds_var=_ds_var,
        model_str="GISTEMP_5x5",
        cesm=False,
        new_times=new_times,
    )

    avail_mask = mask_IPCC_byavailability(
        filepath=_ds_filepath,
        var=_ds_var,
        regions=regionmask.defined_regions.ar6.all,
        masking_threshold=0.9,
        ufunc=gistemp_5x5_preproc,
        new_times=new_times,
    )

    ipcc_regions_maskedavail = xagg_region_means.where(avail_mask)
    
    # Add "realization" coordinate so concatenation is easier.
    ipcc_regions_maskedavail = ipcc_regions_maskedavail.assign_coords(realization=i+1).expand_dims("realization")
    
    filename = _ds_filepath.split('/')[-1]
    
    _outfilepath = '%s/%s/%s' % (save_dir,model_subdir,filename)
    print(_outfilepath)
    
    # break
    ipcc_regions_maskedavail.to_netcdf(path=_outfilepath)
    # if i == 2: break
    
    

/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of the pixels in variable '+var+' have nans in them in the dimensions '+
/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of 

/glade/u/home/jonahshaw/w/trend_uncertainty/nathan/OBS_LENS//GISTEMP_5x5/20240820/xagg_correctedtime//ensembleChunk_5x5_0001.nc


/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of the pixels in variable '+var+' have nans in them in the dimensions '+
/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of 

/glade/u/home/jonahshaw/w/trend_uncertainty/nathan/OBS_LENS//GISTEMP_5x5/20240820/xagg_correctedtime//ensembleChunk_5x5_0002.nc


/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of the pixels in variable '+var+' have nans in them in the dimensions '+
/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of 

...

/glade/u/home/jonahshaw/w/trend_uncertainty/nathan/OBS_LENS//GISTEMP_5x5/20240820/xagg_correctedtime//ensembleChunk_5x5_0198.nc


/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of the pixels in variable '+var+' have nans in them in the dimensions '+
/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of 

/glade/u/home/jonahshaw/w/trend_uncertainty/nathan/OBS_LENS//GISTEMP_5x5/20240820/xagg_correctedtime//ensembleChunk_5x5_0199.nc


/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of the pixels in variable '+var+' have nans in them in the dimensions '+
/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of 

/glade/u/home/jonahshaw/w/trend_uncertainty/nathan/OBS_LENS//GISTEMP_5x5/20240820/xagg_correctedtime//ensembleChunk_5x5_0200.nc
CPU times: user 7min 18s, sys: 4min 54s, total: 12min 12s
Wall time: 12min 41s


#### HadCRUT5

In [18]:
# Arbitrary function to apply

def hadcrut5_preproc(da:xr.DataArray):
    
    return da.rename({'longitude':'lon','latitude':'lat'})

In [19]:
%%time

model_subdir = 'HadCRUT5/20240820/xagg'

if not os.path.exists(os.path.join(save_dir,model_subdir)):
    os.makedirs(os.path.join(save_dir,model_subdir))

# Variable to select and operate over.
_ds_var = hadcrut5_tas_var

for _ds_filepath in hadcrut5_files:

    xagg_region_means = create_ipccregion_timeseries_xagg(
        ds_filepath=_ds_filepath,
        ds_var=_ds_var,
        model_str="HadCRUT5",
        cesm=False,
    )

    avail_mask = mask_IPCC_byavailability(
        filepath=_ds_filepath,
        var=_ds_var,
        regions=regionmask.defined_regions.ar6.all,
        masking_threshold=0.9,
        ufunc=hadcrut5_preproc,
    )

    ipcc_regions_maskedavail = xagg_region_means.where(avail_mask)
    
    filename = _ds_filepath.split('/')[-1]
    _realization = filename.split(".")[-2]

    ipcc_regions_maskedavail = ipcc_regions_maskedavail.assign_coords(realization=_realization).expand_dims("realization")
    
    _outfilepath = '%s/%s/%s' % (save_dir,model_subdir,filename)
    print(_outfilepath)
    
    ipcc_regions_maskedavail.to_netcdf(path=_outfilepath)
    if i == 2: break


/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of the pixels in variable '+var+' have nans in them in the dimensions '+
/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable tas have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of 

/glade/u/home/jonahshaw/w/trend_uncertainty/nathan/OBS_LENS//HadCRUT5/20240820/xagg/HadCRUT.5.0.2.0.analysis.anomalies.1.nc
CPU times: user 3.26 s, sys: 2.44 s, total: 5.7 s
Wall time: 10.7 s


#### DCENT (unfilled)

In [43]:
%%time

model_subdir = 'DCENT/20240901/xagg'

if not os.path.exists(os.path.join(save_dir,model_subdir)):
    os.makedirs(os.path.join(save_dir,model_subdir))

# Variable to select and operate over.
_ds_var = dcent_unfiled_tas_var

for _ds_filepath in dcent_unfilled_files:

    xagg_region_means = create_ipccregion_timeseries_xagg(
        ds_filepath=_ds_filepath,
        ds_var=_ds_var,
        model_str="DCENT_unfilled",
        cesm=False,
    )

    avail_mask = mask_IPCC_byavailability(
        filepath=_ds_filepath,
        var=_ds_var,
        regions=regionmask.defined_regions.ar6.all,
        masking_threshold=0.9,
    )

    ipcc_regions_maskedavail = xagg_region_means.where(avail_mask)
    
    filename = _ds_filepath.split('/')[-1]
    _realization = int(filename.split(".")[-2][-3:])
    
    ipcc_regions_maskedavail = ipcc_regions_maskedavail.assign_coords(realization=_realization).expand_dims("realization")

    _outfilepath = '%s/%s/%s' % (save_dir,model_subdir,filename)
    print(_outfilepath)
    
    break
    ipcc_regions_maskedavail.to_netcdf(path=_outfilepath)
    if i == 2: break


/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable temperature have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn('One or more of the pixels in variable '+var+' have nans in them in the dimensions '+
/glade/work/jonahshaw/conda-envs/py_xagg/lib/python3.12/site-packages/xagg/core.py:724: UserWarning: One or more of the pixels in variable temperature have nans in them in the dimensions time. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
  warnings.warn(

/glade/u/home/jonahshaw/w/trend_uncertainty/nathan/OBS_LENS//DCENT/20240901/xagg/DCENT_ensemble_1850_2023_member_001.nc
CPU times: user 2.09 s, sys: 577 ms, total: 2.67 s
Wall time: 2.67 s
